In [ ]:
import keras
from keras.api.layers import Input, LSTM, Dense, Embedding, Attention, Concatenate
from keras.api.models import Model
from keras.src.legacy.preprocessing.text import Tokenizer
from keras.api.preprocessing.sequence import pad_sequences

print(keras.__version__)
print(Tokenizer)


3.8.0
<class 'keras.src.legacy.preprocessing.text.Tokenizer'>


In [ ]:
import keras
from keras.api.layers import Input, LSTM, Dense, Embedding, Attention, Concatenate
from keras.api.models import Model
from keras.src.legacy.preprocessing.text import Tokenizer
from keras.api.preprocessing.sequence import pad_sequences
import numpy as np

#Load Cornell movie dataset
#Assume questions and answers are a list of strings
tokenizer =  Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(questions + answers)
#convert text to sequences
question_sequences = tokenizer.texts_to_sequences(questions)
answer_sequences = tokenizer.texts_to_sequences(answers)

#pad sequences
max_len = 20
X = pad_sequences(question_sequences, maxlen=max_len, padding='post')
y = pad_sequences(answer_sequences, maxlen=max_len, padding='post')

vocab_size = len(tokenizer.word_index) + 1

#Encoder
encoder_inputs = Input(shape=(max_len,))
enc_emb = Embedding(input_dim=vocab_size, output_dim=256)(encoder_inputs)
encoder_lstm = LSTM(256, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)

#Decoder with Attention
decoder_inputs = Input(shape=(max_len,))
dec_emb = Embedding(input_dim=vocab_size, output_dim=256)(decoder_inputs)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=[state_h, state_c])

#Attention layer
attention = Attention(use_scale=True)([decoder_outputs, encoder_outputs])
decoder_concat = Concatenate()([decoder_outputs, attention])

#Output layer
output = Dense(vocab_size, activation='softmax')(decoder_concat)

#Define Model
model = Model(inputs=[encoder_inputs, decoder_inputs], outputs = output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

#Training
model.fit(
    [X, y[:, :-1]],     #Teacher forcing : decoder input is shifted left
    y[:,1:],              # Target is next word in sequence
    epochs = 30,
    batch_size = 64,
)

#Inference(Prediction Loop)

#Define encoder model
encoder_model = Model(encoder_inputs, [encoder_outputs, state_h, state_c])

#Define decoder model(for step by step prediction)
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_outputs, state_h, state_c = decoder_lstm(
    dec_emb, initial_state=[decoder_state_input_h, decoder_state_input_c]
)
decoder_model = Model(
    [decoder_inputs] + [decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs, state_h,state_c],
)

#Generate Responses
def generate_response(input_text):
  #Tokenize input
  input_seq = tokenizer.texts_to_sequences([input_text])
  input_seq = pad_sequences(input_seq, maxlen=max_len, padding='post')

  #Encode input
  encoder_output, state_h, state_c = encoder_model.predict(input_seq)

  # Start decoding with "<sos>" token
  target_seq = np.zeros((1,1))
  target_seq[0,0] = tokenizer.word_index["<sos>"]

  #Generate response word by word
  response = []
  for _ in range(max_len):
    output, state_h, state_c = decoder_model.predict([target_seq] + [state_h,state_c])
    predicted_word_index = np.argmax(output[0,-1, :])
    if predicted_word_index == tokenizer.word_index["<eos>"]:
      break
    response.append(tokenizer.index_word[predicted_word_index])
    target_seq = np.zeros((1,1))
    target_seq[0,0] = predicted_word_index

    return " ".join(response)
